In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

# Define transforms: convert images to tensor and normalize them
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download and load the CIFAR10 dataset (as an example)
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Input channels 3 (RGB), 32 filters
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),                          # Downsample by factor of 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 128),  # 8x8 is the image size after pooling from 32x32 images
            nn.ReLU(inplace=True),
            nn.Linear(128, 10)           # 10 classes for CIFAR10
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layers
        x = self.classifier(x)
        return x

model = SimpleCNN()


In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()      # Combines softmax and negative log-likelihood loss
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [5]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()            # Clear previous gradients
        outputs = model(images)          # Forward pass
        loss = criterion(outputs, labels)
        loss.backward()                  # Backward pass (compute gradients)
        optimizer.step()                 # Update model parameters
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")


Epoch [1/10], Loss: 1.3512


KeyboardInterrupt: 

In [6]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for inference
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the highest logit
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Accuracy on test set: 63.64%
